In [109]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [91]:
from lib.utils import connect_wifi, load_env_vars

env_vars = load_env_vars("lib/.envh")
# connect WiFI
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

connecting to network...
network config: ('172.20.10.4', '255.255.255.240', '172.20.10.1', '172.20.10.1')


In [98]:
from lib.runner import OnlineRunner
from lib.logging import logger_types

decode_period_s = 4 # decode every x seconds
Nc = 1
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]
api_host = "http://172.20.10.2:5001/" # make sure the port corresponds to your logging server configuration

log_params = dict(server=api_host, 
                  log_period=4, 
                  logger_type=logger_types.HTTP, 
                  send_raw=True, 
                  session_id='hotspot')

runner = OnlineRunner('CCA', buffer_size=Ns)
runner.setup(**log_params)
runner.run()

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
connecting to network...
network config: ('172.20.10.4', '255.255.255.240', '172.20.10.1', '172.20.10.1')


In [100]:
print(runner.output_buffer)
runner.stop()

[-0.04179526, -1.87689, -6.050216, -4.619785, -12.30089, -29.34657, -5.430662, 16.15306, 1.520026, 17.09361, 36.15711, 20.44824, -9.487555, -29.90106, -20.17232, -15.53988, -22.40484, 13.78118, 37.3049, 35.42534, 45.44753, 18.2136, 4.41837, -17.95101, -44.72243, -27.52366, -11.02454, 18.07533, 25.26217, 21.53173, 16.21344, 14.81441, -3.934866, -38.38978, -21.07615, 0.900435, 3.538435, -31.65422, -29.26047, -6.156304, 8.204639, 39.88416, 14.10834, 10.32538, 8.813786, -14.11746, -33.01486, -17.81416, 1.329798, -19.52213, 19.01212, 37.36596, 21.30258, 0.3455695, -27.02314, -29.83115, -23.26448, -1.170373, 21.85099, 28.86534, 6.050815, -15.00704, -33.18097, -20.95881, -0.03471351, -0.9796466, -0.3909593, 4.687751, -6.90556, -29.54284, -27.91251, -14.46558, 14.10185, 32.66275, 16.51373, 7.94457, 6.191947, -12.45231, -38.09886, -15.3043, -3.268488, -18.47307, 16.48532, 36.10726, 20.96221, -2.681967, -26.20954, -33.31726, -22.65532, -3.912046, 21.35466, 27.39192, 4.310492, -15.41527, -35.6973

In [78]:
f = open('lib/.envh', 'r')
print(f.read())

#.envh
WIFI_SSID=Rishil
WIFI_PASSWORD=rishilhotspot



In [106]:
%sendtofile main.py --source main.py

Sent 352 lines (6230 bytes) to main.py.


# Local functionality

In [51]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

counter = 0
while counter !=10:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

p26.off()
p13.off()

time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = runner.output_buffer
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

print(1)

decode_period_s = 4 # decode every x seconds
Nc = 1
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

# while True:
#     if not buttonB.value():
#         p26.on()
#         calibration_data[7] = getData(Nt, decode_period_s)
#         gc.collect()
#         p26.off()
#         break

calibration_data[7] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

print(3)

decode = Runner('MsetCCA', buffer_size=Ns) # initialise a base runner
decode.setup() # setup peripherals and memory buffers
decode.calibrate(calibration_data)

print(4)

decode.run()

print(5)

for i in range(5):
    p13.on()
    time.sleep(5)
    print(decode.decoded_output)
    p13.off()

print(6)
decode.stop()
print(7)

.

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 30, in <module>
KeyboardInterrupt: 


# Local with preprocess

In [110]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

counter = 0
while counter !=10:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

p26.off()
p13.off()

time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def preprocess_data(signal):
    
    """Preprocess incoming signal before decoding algorithms.
    This involves applying a bandpass filter to isolate the target SSVEP range
    and then downsampling the signal to the Nyquist boundary.
    
    Returns:
        [np.ndarray]: filtered and downsampled signal
    """
    from lib.signal import sos_filter
    downsample_freq = 64
    ds_factor = 256//downsample_freq
    return sos_filter(signal)[::ds_factor]

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = preprocess_data(runner.output_buffer)
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

print(1)

decode_period_s = 4 # decode every x seconds
Nc = 1
Ns = 256
Nt = 3
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

# while True:
#     if not buttonB.value():
#         p26.on()
#         calibration_data[7] = getData(Nt, decode_period_s)
#         gc.collect()
#         p26.off()
#         break

calibration_data[7] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

print(3)

decode = Runner('MsetCCA', buffer_size=Ns) # initialise a base runner
decode.setup() # setup peripherals and memory buffers
decode.calibrate(calibration_data)

print(4)

decode.run()

print(5)

for i in range(5):
    p13.on()
    time.sleep(5)
    print(decode.decoded_output)
    p13.off()

print(6)
decode.stop()
print(7)

....1
ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
2
....62688
66096
.....61872
65280
....61040
64448
...3
ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
4
5
Traceback (most recent call last):
  File "lib/runner.py", line 139, in sample_callback
  File "lib/runner.py", line 104, in decode
  File "lib/decoding.py", line 237, in classify
  File "lib/decoding.py", line 41, in compute_corr
  File "lib/decoding.py", line 145, in cca_eig
ValueError: dimensions do not match
Traceback (most recent call last):
  File "lib/runner.py", line 139, in sample_callback
  File "lib/runner.py", line 104, in decode
  File "lib/decoding.py", line 237, in classify
  File "lib/decoding.py", line 41, in compute_corr
  File "lib/decoding.py", line 145, in cca_eig
ValueError: dimensions do not match
.Traceback (most recent call last):
  File "lib/runner.py", line 139, in sample_callback
  File "lib/runner.py", line 104, in decode
  File "lib/decoding.py", line 23

Traceback (most recent call last):
  File "<stdin>", line 176, in <module>
KeyboardInterrupt: 


In [111]:
print(calibration_data)

[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "lib/runner.py", line 139, in sample_callback']
[leftinbuffer] ['  File "lib/runner.py", line 104, in decode']
[leftinbuffer] ['  File "lib/decoding.py", line 237, in classify']
[leftinbuffer] ['  File "lib/decoding.py", line 41, in compute_corr']
[leftinbuffer] ['  File "lib/decoding.py", line 145, in cca_eig']
[leftinbuffer] ['ValueError: dimensions do not match']
[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "lib/runner.py", line 139, in sample_callback']
[leftinbuffer] ['  File "lib/runner.py", line 104, in decode']
[leftinbuffer] ['  File "lib/decoding.py", line 237, in classify']
[leftinbuffer] ['  File "lib/decoding.py", line 41, in compute_corr']
[leftinbuffer] ['  File "lib/decoding.py", line 145, in cca_eig']
[leftinbuffer] ['ValueError: dimensions do not match']
[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "lib/runner.py", line 139, i

# Online Functionality

In [102]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

[leftinbuffer] ['ets Jul 29 2019 12:21:46']
[leftinbuffer] ['rst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)']
[leftinbuffer] ['configsip: 0, SPIWP:0xee']
[leftinbuffer] ['clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00']
[leftinbuffer] ['mode:DIO, clock div:2']
[leftinbuffer] ['load:0x3fff0018,len:4']
[leftinbuffer] ['load:0x3fff001c,len:5204']
[leftinbuffer] ['load:0x40078000,len:12136']
[leftinbuffer] ['load:0x40080400,len:3496']
[leftinbuffer] ['entry 0x4008063c']
Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [103]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

counter = 0
while counter !=10:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

p26.off()
p13.off()

time.sleep(5)

from lib.utils import connect_wifi, load_env_vars

# env_vars = load_env_vars("lib/.envh")
# # connect WiFI
# ssid = env_vars.get("WIFI_SSID")
# password = env_vars.get("WIFI_PASSWORD")
# ssid ="Rishil"
# password = "rishilhotspot"
# connect_wifi(ssid, password)

# # import webrepl
# # webrepl.start()

p26.on()
p13.on()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = runner.output_buffer
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

print(1)

decode_period_s = 4 # decode every x seconds
Nc = 1
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

calibration_data[7] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

print(3)

del runner

from lib.runner import OnlineRunner
from lib.logging import logger_types

api_host = "http://172.20.10.2:5001/" # make sure the port corresponds to your logging server configuration

log_params = dict(server=api_host, 
                  log_period=4, 
                  logger_type=logger_types.HTTP, 
                  send_raw=True, 
                  session_id='hotspot_test_wireless')

runner = OnlineRunner('MsetCCA', buffer_size=Ns)
runner.setup(**log_params)

runner.calibrate(calibration_data)
runner.run()


....1
ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
2
....60992
67728
.....59408
66144
....57808
64544
...3
ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
connecting to network...
..network config: ('172.20.10.4', '255.255.255.240', '172.20.10.1', '172.20.10.1')


In [105]:
runner.stop()

In [104]:
print(runner.is_sampling)

True


# Online with downsample

In [ ]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

In [ ]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

counter = 0
while counter !=10:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

p26.off()
p13.off()

time.sleep(5)

from lib.utils import connect_wifi, load_env_vars

# env_vars = load_env_vars("lib/.envh")
# # connect WiFI
# ssid = env_vars.get("WIFI_SSID")
# password = env_vars.get("WIFI_PASSWORD")
# ssid ="Rishil"
# password = "rishilhotspot"
# connect_wifi(ssid, password)

# # import webrepl
# # webrepl.start()

p26.on()
p13.on()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def preprocess_data(signal):
    
    """Preprocess incoming signal before decoding algorithms.
    This involves applying a bandpass filter to isolate the target SSVEP range
    and then downsampling the signal to the Nyquist boundary.
    
    Returns:
        [np.ndarray]: filtered and downsampled signal
    """
    from lib.signal import sos_filter
    downsample_freq = 64
    ds_factor = 256//downsample_freq
    return sos_filter(signal)[::ds_factor]

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = preprocess_data(runner.output_buffer)
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

print(1)

decode_period_s = 4 # decode every x seconds
Nc = 1
Ns = 256
Nt = 3
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

calibration_data[7] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

print(3)

del runner

from lib.runner import OnlineRunner
from lib.logging import logger_types

api_host = "http://172.20.10.2:5001/" # make sure the port corresponds to your logging server configuration

log_params = dict(server=api_host, 
                  log_period=4, 
                  logger_type=logger_types.HTTP, 
                  send_raw=True, 
                  session_id='hotspot_test_wireless')

runner = OnlineRunner('MsetCCA', buffer_size=Ns)
runner.setup(**log_params)

runner.calibrate(calibration_data)
runner.run()
